In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
#import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
#import seaborn as sns

import keras_tuner
import tensorflow as tf
import tensorflow.keras.layers as tfl

from sklearn.metrics import confusion_matrix, classification_report

2022-12-17 14:32:43.130837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
